In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
df.dropna(inplace=True)
df.columns = ['status', 'stay-time', 'Aspect-ratio', 'Density', 'Dis Variance', 'Velocity Mean', 'Velocity Variance',
              'a-Variance', 'maxDis', 'head-Variance', 'Class label']
df.head(5)

,status,stay-time,Aspect-ratio,Density,Dis Variance,Velocity Mean,Velocity Variance,a-Variance,maxDis,head-Variance,Class label
0,1.0,5.520556,8.594854,0.024205,3.278965,0.014234,0.008460,0.000056,167.689320,32.164128,1
1,1.0,3.850833,3.030676,0.051397,5.937132,0.034289,0.027529,0.000134,66.725869,5.636387,1
2,1.0,3.850556,6.788039,0.041447,3.433945,0.023077,0.014594,0.000103,110.791868,7.544310,1
3,1.0,4.348889,6.200071,0.011759,7.810434,0.029176,0.024673,0.000097,192.090427,9.990709,1
4,1.0,2.199444,2.410629,0.044884,5.714460,0.033267,0.025093,0.000141,45.923765,4.390358,1


In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
X = df.iloc[:, 0:10].values
y = df.iloc[:, 10].values
seed = 0
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
model = DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, min_samples_split=2).fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy)


print('***********')
scores = cross_val_score(model, X_train, y_train, cv=10)
print(scores)  # 打印输出每次迭代的度量值（准确度）
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))  # 获取置信区间。（也就是均值和方差）
print()
scores = cross_val_score(model, X_train, y_train, cv=10)  
print('准确率：',np.mean(scores), scores)  
precisions = cross_val_score(model, X_train, y_train, cv=10, scoring='precision')  
print('精确率：', np.mean(precisions), precisions)  
recalls = cross_val_score(model, X_train, y_train, cv=10, scoring='recall')  
print('召回率：', np.mean(recalls), recalls)  
f1s = cross_val_score(model, X_train, y_train, cv=10, scoring='f1')  
print('综合评价指标：', np.mean(f1s), f1s)  

#################################################

0.9481006271174224
***********
[0.94966028 0.94811612 0.95427865 0.94964473 0.94840902 0.95767686
 0.94531974 0.94810009 0.94962917 0.94530284]
Accuracy: 0.950 (+/- 0.007)

准确率： 0.9496137521040978 [0.94966028 0.94811612 0.95427865 0.94964473 0.94840902 0.95767686
 0.94531974 0.94810009 0.94962917 0.94530284]
精确率： 0.953251698331967 [0.95424476 0.9541183  0.95264017 0.95125958 0.95615982 0.9548913
 0.94693654 0.95162177 0.95826377 0.95238095]
召回率： 0.9569244427089213 [0.95582551 0.95306461 0.96631695 0.9591386  0.95140806 0.97018222
 0.95582551 0.95582551 0.9519337  0.94972376]
综合评价指标： 0.9551286335735524 [0.95532267 0.95388014 0.95942982 0.95518284 0.95377802 0.96247603
 0.95136026 0.95371901 0.95481009 0.95132743]


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
X = df.iloc[:,0:10]
Y = df.iloc[:,10]
model = DecisionTreeClassifier(random_state=0)
param_grid = dict(max_depth=[6], min_samples_split=[2], min_samples_leaf=[8])
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) #n_jobs: 并行数，int：个数,-1：跟CPU核数一致, 1:默认值
grid_result = grid_search.fit(X_train, y_train)
pred = grid_result.predict(X_test)
print("最优测试集验证得分:"+str(np.mean(pred == y_test)))

#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#Best: -0.197524 using {'max_depth': 9, 'min_samples_leaf': 7, 'min_samples_split': 0.1}

最优测试集验证得分:0.9480285446550855


In [3]:
# Tune learning_rate
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
# load data
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
# split data into X and y
X = df.iloc[:,0:10]
Y = df.iloc[:,10]
# grid search
model = DecisionTreeClassifier()
#param_grid = dict(learning_rate=learning_rate, n_estimators=[10, 50, 100], max_depth=np.arange(3, 8), min_child_weight=np.arange(3, 8), subsample=np.arange(5, 9)/10)
param_grid = dict(max_depth=np.arange(5,10), min_samples_split=np.arange(2,10,1), min_samples_leaf=np.arange(3,10,1))
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) #n_jobs: 并行数，int：个数,-1：跟CPU核数一致, 1:默认值
grid_result = grid_search.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with: %r" % (mean, param))

ValueError: Invalid parameter n_estimators for estimator DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'). Check the list of available parameters with `estimator.get_params().keys()`.